In [73]:
# Flops in Transformer
# per:
# Scaling Laws for Neural Language Models
# https://arxiv.org/pdf/2001.08361.pdf

n_layer      = 4     # number of layers
d_model      = 256   # dimension of residual stream
d_ff         = 4 * d_model   # dimension of intermediate feed-forward layer
d_attn       = 256   # dimension of the attention output
n_heads      = 4     # number of attention heads per layer
n_ctx        = 128   # number of context input tokens
n_vocab      = 6064  # vocab size
d_embd       = 256   # embedding size

# GPT-1
#n_layer      = 12    # number of layers
#d_model      = 768   # dimension of residual stream
#d_ff         = 3072  # dimension of intermediate feed-forward layer
#d_attn       = 768   # dimension of the attention output
#n_heads      = 12    # number of attention heads per layer
#n_ctx        = 256   # number of context input tokens
#n_vocab      = 40000 # vocab size
#d_embd       = 768   # embedding size

# GPT-2
#n_layer      = 48    # number of layers
#d_model      = 1600  # dimension of residual stream
#d_ff         = 3072  # dimension of intermediate feed-forward layer
#d_attn       = 1600  # dimension of the attention output
#n_heads      = 48    # number of attention heads per layer
#n_ctx        = 1024  # number of context input tokens
#n_vocab      = 50257 # vocab size
#d_embd       = 1600  # embedding size

# GPT-3
#n_layer      = 96    # number of layers
#d_model      = 12288 # dimension of residual stream
#d_ff         = 4 * d_model  # dimension of intermediate feed-forward layer
#d_attn       = 12288 # dimension of the attention output
#n_heads      = 96    # number of attention heads per layer
#n_ctx        = 2048  # number of context input tokens
#n_vocab      = 50257 # vocab size
#d_embd       = 12288 # embedding size

#         self.ln1 = nn.LayerNorm(config.n_embd)
#         self.ln2 = nn.LayerNorm(config.n_embd)
#         self.attn = CausalSelfAttention(config)
#         self.mlp = nn.Sequential(
#             nn.Linear(config.n_embd, 4 * config.n_embd),
#             nn.GELU(),
#             nn.Linear(4 * config.n_embd, config.n_embd),
#             nn.Dropout(config.resid_pdrop),
#         )
        
# attn k 256 * 256
# attn q 256 * 256
# attn v 256 * 256
# attn p 256 * 256
# mlp 0 - 4 * 256 * 256
# mlp 2 - 4 * 256 * 256

params = {
    'embed':     (n_vocab + n_ctx) * d_model,
    'attn_qkv':  n_layer * d_model * 3 * d_attn,
    'attn_mask': 0,
    'attn_proj': n_layer * d_model * d_attn,
    'ff':        n_layer * 2 * d_model * d_ff,
    'de-embed':  0,
    'dec_head':  (n_vocab) * d_model
}

# FLOPs per token
flops = {
    'embed':     4 * d_model,
    'attn_qkv':  2 * n_layer * d_model * 3 * d_attn,
    'attn_mask': 2 * n_layer * n_ctx * d_attn,
    'attn_proj': 2 * n_layer * d_attn * d_embd,
    'ff':        2 * n_layer * 2 * d_model * d_ff,
    'de-embed':  2 * d_model * n_vocab,
}


total_params = 0
print('params\n-----')
for k in params:
    print(f'{k:11s}: {params[k]:12d}')
    total_params += params[k]
print(f'params (G) : {total_params / (1e9) :12.6f}')

total_flops = 0
print('\nflops\n-----')
for k in flops:
    print(f'{k:11s}: {flops[k]:12d}')
    total_flops += flops[k]

print(f'total GF   : {total_flops / (1e9) :12.6f}')
print('')

#  the number of model parameters, excluding all vocabulary and positional embeddings
N = 2 * d_model * n_layer * (2 * d_attn + d_ff)
# total, non-embedding
C_forward = 2 * N + 2 * n_layer * n_ctx * d_model # 2.2
print(f'{C_forward=}')

params
-----
embed      :      1585152
attn_qkv   :       786432
attn_mask  :            0
attn_proj  :       262144
ff         :      2097152
de-embed   :            0
dec_head   :      1552384
params (G) :     0.006283

flops
-----
embed      :         1024
attn_qkv   :      1572864
attn_mask  :       262144
attn_proj  :       524288
ff         :      4194304
de-embed   :      3104768
total GF   :     0.009659

C_forward=6553600
